In [ ]:
# ЧЕРНОВОЙ ВАРИАНТ ПО УДАЛЕНИЮ ДУБЛЕЙ С ПОМОЩЬЮ AUTO KERAS CLASSIFICATION В COLAB

# AUTOKERAS TUTORIAL https://autokeras.com/tutorial/text_classification/

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf "/content/drive/MyDrive/KIA_DATASET/AUTO_KERAS/"
!mkdir -p "/content/drive/MyDrive/KIA_DATASET/AUTO_KERAS/"

In [ ]:
KEY="04_TRAIN"
ALL_TRAIN_TEXTS={
  "04_TRAIN": "https://drive.google.com/file/d/12B7YYHE8PsS2om0RTtiou5AunBMzZyJ2/"
, "03_TEST0": "https://drive.google.com/file/d/1Y-ayV4rKHKCrtDe-IdYtxyNCcWnktWnl/"
, "02_TEST1": "https://drive.google.com/file/d/1sYXxm0VMPqAm7Md7FCys5IFWebjYdbCu/"
, "01_TEST2": "https://drive.google.com/file/d/1NQMl6wToKE-Xb-Y2jTSMadnLUnY7oKlu/"
}

In [ ]:
# Переводим текстовые файл в csv для train
import requests
import re
import time

def convert_txt_to_train_csv(url, output_path):
    match_ = re.search('/file/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    response = requests.get("https://docs.google.com/uc?export=download&id="+doc_id)
    encoding = response.encoding
    content = response.text.encode(encoding).decode('utf-8')
    print(content.split("\n")[0])
    print("...")
    return content

BASE = "/content/drive/MyDrive/KIA_DATASET/AUTO_KERAS/"

with open(BASE + 'dataset.csv', 'w') as fw:
    for key in sorted(ALL_TRAIN_TEXTS.keys()):
        time.sleep(5) # DOn'T LET GOOGLE BAN US
        for line in convert_txt_to_train_csv(ALL_TRAIN_TEXTS[key], BASE + "dataset.csv").split("\n"):
            fw.write(line.replace('?|||POS_TEST','0|||TRAIN').replace('?|||NEG_TEST','1|||TRAIN').replace('\t', '   ').replace('|||', '\t') + "\n")

?|||POS_TEST|||Техническая информация|||узнать о возможности установки датчика слепых зон на машину Kia Селтос|||Клиент спрашивает о возможности установки датчика слепых зон на машину Kia Селтос. Оператор отвечает, что установка дополнительного оборудования входит в компетенцию дилерских центров и рекомендует обратиться в официальный дилерский центр Kia для получения консультации по наличию и стоимости опций и аксессуаров. Оператор также предоставляет ссылку на список и координаты всех официальных дилеров Kia. Оператор спрашивает, нужна ли еще помощь клиенту. Оператор благодарит клиента за обращение.|||Техническая информация|||узнать о возможности установки датчика слепых зон на машину Kia Селтос|||Клиент спрашивает о возможности установки датчика слепых зон на машину Kia Селтос. Оператор отвечает, что установка дополнительного оборудования входит в компетенцию дилерских центров и рекомендует обратиться в официальный дилерский центр Kia для получения консультации по наличию и стоимости

In [ ]:
#@title Установка библиотек
!pip install nltk autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.1/590.1 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.2 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:


In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pickle
from nltk import ngrams

def text_classification_train():
    # Загрузка данных
    df = pd.read_csv(BASE + 'dataset.csv', delimiter='\t', header=None)

    # Replacing infinite with nan
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Dropping all the rows with nan values
    df.dropna(inplace=True)

    # Разделение данных на тексты и метки
    texts_2_4 = [' '.join(c) for c in df.iloc[:, 2:4].values]
    texts_5_7 = [' '.join(c) for c in df.iloc[:, 5:7].values]
    texts_8_10 = [' '.join(c) for c in df.iloc[:, 8:10].values]
    texts = [' '.join(c) for c in df.iloc[:, 1:10].values]
    labels = df.iloc[:, 0].values

    # Создание и обучение модели с помощью auto_keras
    # import autokeras as ak
    # clf = ak.TextClassifier(overwrite=True, max_trials=3)
    # clf.fit(texts, labels, epochs=10)
    # clf.export_autokeras_model(BASE + 'ak_text_classification_model.h5')

    # Токенизация текстов
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    sequences_2_4 = tokenizer.texts_to_sequences(texts_2_4)
    sequences_5_7 = tokenizer.texts_to_sequences(texts_5_7)
    sequences_8_10 = tokenizer.texts_to_sequences(texts_8_10)
    sequences = tokenizer.texts_to_sequences(texts)

    # Приведение текстов к единой длине
    max_sequence_length = 2 * max([len(seq) for seq in sequences])
    padded_sequences_2_4 = pad_sequences(sequences_2_4, maxlen=max_sequence_length)
    padded_sequences_5_7 = pad_sequences(sequences_5_7, maxlen=max_sequence_length)
    padded_sequences_8_10 = pad_sequences(sequences_8_10, maxlen=max_sequence_length)
    padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

    # README http://www.dwbiadda.com/keras-text-preprocessing-and-image-preprocessing/

    # Разделение на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.02, random_state=42)

    # import autokeras as ak
    # clf = ak.ImageClassifier()
    # clf.fit(padded_sequences, labels)
    # clf.export_autokeras_model(BASE + 'ak_text_classification_model.h5')

    # Создание модели
    model = keras.Sequential([
        keras.layers.Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_sequence_length),
        keras.layers.GlobalAveragePooling1D(),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция и обучение модели
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
    print("ACCURACY: " + str(history.history['accuracy']) + " !!!!!")
    if history.history['accuracy'][-1] < 0.51 and history.history['accuracy'][-1] < 0.49:
        print("""raise Exception("STOP!!!! BAD ACCURACY 50/50")""")

    # Сохранение модели на диск
    model.save(BASE + 'user_text_classification_model.h5')
    with open(BASE + 'tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    return [BASE + 'user_text_classification_model.h5', BASE + 'user_text_classification_model.h5', BASE + 'tokenizer.pickle', max_sequence_length]

saved = text_classification_train()

Epoch 1/10
57/57 [==============================] - 4s 42ms/step - loss: 0.6910 - accuracy: 0.5164 - val_loss: 0.6899 - val_accuracy: 0.5676
Epoch 2/10
57/57 [==============================] - 2s 34ms/step - loss: 0.6817 - accuracy: 0.6613 - val_loss: 0.6777 - val_accuracy: 0.6486
Epoch 3/10
57/57 [==============================] - 2s 30ms/step - loss: 0.6602 - accuracy: 0.6981 - val_loss: 0.6452 - val_accuracy: 0.6216
Epoch 4/10
57/57 [==============================] - 2s 41ms/step - loss: 0.6239 - accuracy: 0.7131 - val_loss: 0.6004 - val_accuracy: 0.6486
Epoch 5/10
57/57 [==============================] - 2s 41ms/step - loss: 0.5844 - accuracy: 0.7287 - val_loss: 0.5944 - val_accuracy: 0.6216
Epoch 6/10
57/57 [==============================] - 3s 44ms/step - loss: 0.5530 - accuracy: 0.7331 - val_loss: 0.5723 - val_accuracy: 0.6486
Epoch 7/10
57/57 [==============================] - 2s 42ms/step - loss: 0.5363 - accuracy: 0.7426 - val_loss: 0.5828 - val_accuracy: 0.6486
Epoch 8/10
57

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Валидирующая выборка будет на всех возможных тройках Диалог1, Диалог1, Диалог2, а на выходе список, что оставляем и что удаляем.

In [ ]:
# Переводим текстовые файл в csv для valid
import requests
import re
import time
from io import StringIO
from keras.models import load_model

class Validator:
    def __init__(self, ak_model_filename, user_model_filename, tokenizer_filename, max_seq_len):
        self.model = load_model(ak_model_filename)
        self.user_model = load_model(user_model_filename)
        with open(tokenizer_filename, 'rb') as handle:
            self.tokenizer = pickle.load(handle)
        self.max_sequence_length = max_seq_len

    def text_classification_check(self, csv_line):
        # Convert String into StringIO
        csv_io = StringIO(csv_line + "\n")
        df = pd.read_csv(csv_io, sep="\t", header=None)
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.dropna(inplace=True)
        texts = [' '.join(c) for c in df.iloc[:, 1:10].values]
        sequences = self.tokenizer.texts_to_sequences(texts)
        # max_sequence_length = max([len(seq) for seq in sequences])
        padded_sequences = pad_sequences(sequences, maxlen=self.max_sequence_length)
        X_valid = padded_sequences
        predictions = self.model.predict(X_valid)
        return int(predictions[0])


In [ ]:
validator = Validator(saved[0], saved[1], saved[2], saved[3])
DEL_MAP = {}
with open(BASE + 'dataset.csv', 'r') as fr:
    all_lines = fr.readlines()
    i = 0
    for i_line in all_lines:
        if i_line.endswith("...\n"):
            i = i + 1
            # print(i_line)
            first_debug = True
            j = 0
            for j_line in all_lines:
                if j_line.endswith("...\n"):
                    j = j + 1
                    # print(j_line)
                    if i < j and j not in DEL_MAP:
                        check_line='\t'.join(["?", "CHECK", i_line.split('\t')[2], i_line.split('\t')[3], i_line.split('\t')[4]
                                              , i_line.split('\t')[2], i_line.split('\t')[3], i_line.split('\t')[4]
                                              , j_line.split('\t')[2], j_line.split('\t')[3], j_line.split('\t')[4]])
                        if first_debug:
                            print(check_line)
                            first_debug = False
                        res = validator.text_classification_check(check_line)
                        if res == 0:
                            print("\n=== DELETE ===")
                            # DELETE last element and break
                            DEL_MAP[j] = [i, check_line]
                            break
                        print("\n=== SKIP ===")


?	CHECK	Техническая информация	узнать о возможности установки датчика слепых зон на машину Kia Селтос	Клиент спрашивает о возможности установки датчика слепых зон на машину Kia Селтос. Оператор отвечает, что установка дополнительного оборудования входит в компетенцию дилерских центров и рекомендует обратиться в официальный дилерский центр Kia для получения консультации по наличию и стоимости опций и аксессуаров. Оператор также предоставляет ссылку на список и координаты всех официальных дилеров Kia. Оператор спрашивает, нужна ли еще помощь клиенту. Оператор благодарит клиента за обращение.	Техническая информация	узнать о возможности установки датчика слепых зон на машину Kia Селтос	Клиент спрашивает о возможности установки датчика слепых зон на машину Kia Селтос. Оператор отвечает, что установка дополнительного оборудования входит в компетенцию дилерских центров и рекомендует обратиться в официальный дилерский центр Kia для получения консультации по наличию и стоимости опций и аксессуа

In [ ]:
print(str(len(DEL_MAP.keys())))
with open(BASE + "result_delete_list.csv", "w") as fd:
    for d in sorted(DEL_MAP.keys()):
        fd.write("DELETE " + str(d) + "\t" + DEL_MAP[d][1].split('\t')[-3] + "\t" + DEL_MAP[d][1].split('\t')[-2] + "\t" + DEL_MAP[d][1].split('\t')[-1] + "\t" + "FAIL_" + '_'.join(DEL_MAP[d][1].split('\t')).replace(" ", "_") + "\n")

905


In [ ]:
print(str(i-len(DEL_MAP.keys())))
with open(BASE + "result_skip_list.csv", "w") as fs:
    k = 0
    for k_line in all_lines:
        if k_line.endswith("...\n"):
            k = k + 1
            # print(k_line)
            if k not in DEL_MAP:
                fs.write("SKIP " + str(k) + "\t" + k_line.split('\t')[-6] + "\t" + k_line.split('\t')[-5] + "\t" + k_line.split('\t')[-4] + "\t" + "OK" + "\n")


1
